IMPORTANT for this build and at end of September 2022 you need branch 2.8 of tensorflow, Bazel 4.2.1 and Python 3.7

This flow follows the procedure [here](https://www.tensorflow.org/lite/android/lite_build ) with script [commands](https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/tools/dockerfiles/tflite-android.Dockerfile).

In [ ]:
# Check Python's version and path.
!python --version
!which python3

In [ ]:
%cd ~
%cd /
!ls

In [ ]:
# Make folder.
!mkdir -p '/android/sdk/cmdline-tools'
# Install jdk.
!apt-get install -y --no-install-recommends default-jdk

In [ ]:
!ls

SDK

In [ ]:
# Install android SDK.
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-6858069_latest.zip
!unzip commandlinetools-linux-6858069_latest.zip -d /tmp
!mv /tmp/cmdline-tools /android/sdk/cmdline-tools/latest
!rm commandlinetools-linux-6858069_latest.zip

Set paths.

In [ ]:
%env PATH = /usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/android/sdk/tools:/android/sdk/platform-tools:/android/android-sdk_linux:/android/sdk/cmdline-tools/cmdline-tools/latest/bin:/android/sdk/platform-tools:/android/ndk:/android/ndk/android-ndk-r19c
%env ANDROID_API_LEVEL=23
#%env ANDROID_SDK_API_LEVEL=29
%env ANDROID_NDK_API_LEVEL=21
%env ANDROID_SDK_HOME /android/sdk
# Build Tools Version liable to change.
%env ANDROID_BUILD_TOOLS_VERSION=30.0.0
%env ANDROID_NDK_HOME=/android/ndk/android-ndk-r19c
# path for libtensorflow_framework.so.2 /usr/local/lib/python3.7/dist-packages/tensorflow/libtensorflow_framework.so.2
# adding it to LD_LIBRARY_PATH.
%env LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/lib/python3.7/dist-packages/tensorflow
!export -p

In [ ]:
%cd ~
%cd /

In [ ]:
%cd /android/sdk/cmdline-tools/latest/bin
!ls

In [ ]:
# Download additional Android tools and libraries (note that you may need to accept the license).
!bash sdkmanager  \
  "build-tools;30.0.0" \
  "platform-tools" \
  "platforms;android-23"

NDK

In [ ]:
# Install android NDK.
!mkdir -p '/android/ndk/'
!wget -q https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip
!unzip android-ndk-r19c-linux-x86_64.zip -d /android
!rm android-ndk-r19c-linux-x86_64.zip
!bash -c "ln -s /android/android-ndk-* /android/ndk"

In [ ]:
# Make all the folder executable.
!chmod -R go=u '/android'

In [ ]:
%cd ~
%cd /
!ls

In [ ]:
# Clone TensorFlow repository.
!git clone https://github.com/tensorflow/tensorflow.git

In [ ]:
# Rename it
!mv /tensorflow /tensorflow_src

In [ ]:
# Checkout branch r2.8 for this build.
%cd tensorflow_src
%cd tensorflow
!git checkout r2.8

In [ ]:
# Based on the above branch Baselisk will download and install appropriate version of Bazel.

# Download latest version of Bazelisk.
!wget https://github.com/bazelbuild/bazelisk/releases/latest/download/bazelisk-linux-amd64

# Make script executable.
!chmod +x bazelisk-linux-amd64

# Adding to the path.
!sudo mv bazelisk-linux-amd64 /usr/local/bin/bazel

# Extract bazel info
!bazel

In [ ]:
!bazel version

In [ ]:
%cd ../
!ls

In [ ]:
# Run configure
# parameters would be '/usr/bin/python3''/usr/local/lib/python3.7/dist-packages''N''N''N''-Wno-sign-compare''y'
!python configure.py


Build TensorFlow Lite

IMPORTANT = By building below then inside android project you will need to get the tensorflow-lite-api.aar file from [here](https://repo1.maven.org/maven2/org/tensorflow/tensorflow-lite-api/2.8.0/).

In [ ]:
!bazel build -c opt --fat_apk_cpu=x86,x86_64,arm64-v8a,armeabi-v7a \
  --host_crosstool_top=@bazel_tools//tools/cpp:toolchain \
  //tensorflow/lite/java:tensorflow-lite

Build TF-Lite.aar and select-tf-ops.aar if tf ops are present.

(no tensorflow-lite-api.aar file needed to be downloaded)

By the time this colab was created a full set of archs was necessary i.e x86,x86_64,arm64-v8a,armeabi-v7a

WARNING = The procedure to generate the tensorflow-lite-select-tf-ops.aar requires high number of CPUs and high RAM. You should consider connecting this colab to a Google Cloud VM instance with minimum of 24 CPUs and 100MB of RAM and the procedure will take 1,5 hours. If you don't then the default runtime will be terminated after 6 hours and the procedure will not have ended.

In [ ]:
!bash tensorflow/lite/tools/build_aar.sh \
  --input_models=/conv_model.tflite \
  --target_archs=x86,x86_64,arm64-v8a,armeabi-v7a